## TASKS
#### 1. Find the details of the restaurant that has the facility of a “book table” before.
#### 2. Make a list of restaurants according to the type of restaurant and location of the restaurants.
#### 3. List the top ten restaurants with the highest rating.
#### 4. Find the average, maximum, and minimum votes grouped by location.
#### 5. Change rate to float type by removing ‘/5’.
#### 6. Find the correlation between cost and rating of the restaurants.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Apache PySpark Final Project-Zomato").getOrCreate()

22/11/03 18:26:50 WARN Utils: Your hostname, pallavi-xps resolves to a loopback address: 127.0.1.1; using 192.168.1.81 instead (on interface wlp2s0)
22/11/03 18:26:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/03 18:26:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/03 18:26:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
zomato_df = spark.read.csv('data/cleaned_zomato_dataset_final.csv',inferSchema=True,header=True)

In [4]:
zomato_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- votes: integer (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- approx_cost_two_people: string (nullable = true)



In [6]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

zomato_df = zomato_df.withColumn('approx_cost_two_people', col('approx_cost_two_people').cast(IntegerType()))

# SOLUTIONS

## 1. Find the details of the restaurant that has the facility of a “book table” before.

In [7]:
restaurant_names_df = zomato_df.filter(col('book_table') == 'Yes').select(col('name').alias("Restaurant Name"),\
                      col('location').alias("Location"), col('phone').alias("Contact Number/s"),\
                      col('type').alias('Restaurant Type'), col('rating').alias('Rating'),\
                      col('cuisines').alias("Cuisines Available"), col('dish_liked').alias('Most Liked Dishes'),\
                      col('book_table').alias('Table Booking Facility?'))

restaurant_names_df.show(5)

+------------------+------------------+----------------+---------------+------+--------------------+--------------------+-----------------------+
|   Restaurant Name|          Location|Contact Number/s|Restaurant Type|Rating|  Cuisines Available|   Most Liked Dishes|Table Booking Facility?|
+------------------+------------------+----------------+---------------+------+--------------------+--------------------+-----------------------+
|      Cafe Shuffle|      Banashankari|  +91 9742166777|           Cafe| 4.2/5|Cafe, Italian, Co...|Mocktails, Peri F...|                    Yes|
|  The Coffee Shack|      Banashankari|  +91 9731644212|           Cafe| 4.2/5|Cafe, Chinese, Co...|Coffee, Spaghetti...|                    Yes|
|          Goa 0 Km|Kumaraswamy Layout|    080 49653548|  Casual Dining| 3.6/5|Goan, Seafood, No...|Sea Food, Crab Me...|                    Yes|
|Sri Udupi Food Hub|         Jayanagar|  +91 9916866033|  Casual Dining| 4.1/5|South Indian, Nor...|Filter Coffee, Ma...|   

In [8]:
restaurant_names_df.toPandas().to_csv('zomato_output_csv/1.Restaurants_with_book_table_facility.csv', index=False)

## 2. Make a list of restaurants according to the type of restaurant and location of the restaurants.

In [14]:
import pyspark.sql.functions as f
from pyspark.sql.functions import collect_set
from pyspark.sql import Window as W  

window_spec = W.partitionBy('type', 'location')

restaurants_list = zomato_df\
                    .withColumn('restaurant list', collect_set('name').over(window_spec))\
                    .select('type', 'location', 'restaurant list')\
                    .distinct().dropna()

restaurants_list.show()

+------------+--------------------+--------------------+
|        type|            location|     restaurant list|
+------------+--------------------+--------------------+
|      Bakery|                 BTM|[Sweet Truth, Jus...|
|      Bakery|           Banaswadi|[The Gluten Free ...|
|      Bakery|   Bannerghatta Road|[Amma's Pastries,...|
|      Bakery|        Basavanagudi|[Amande Patisseri...|
|      Bakery|         Frazer Town|[Atty's Bakery, T...|
|      Bakery|          HBR Layout|   [Amma's Pastries]|
|      Bakery|                 HSR|[Just Bake, L J I...|
|      Bakery|ITPL Main Road, W...|           [Cakesta]|
|      Bakery|         Indiranagar|    [Karachi Bakery]|
|      Bakery|            JP Nagar|[Bake Addiction, ...|
|      Bakery|  Jeevan Bhima Nagar|         [Just Bake]|
|      Bakery|        Kalyan Nagar|         [Just Bake]|
|      Bakery|Koramangala 1st B...|           [Bakingo]|
|      Bakery|Koramangala 5th B...|         [Just Bake]|
|      Bakery|        Malleshwa

In [15]:
restaurants_list.toPandas().to_csv('zomato_output_csv/2.Restaurants_list_acc_to_type_location.csv', index=False)

## 3. List the top ten restaurants with the highest rating

In [16]:
from pyspark.sql.functions import desc

top_rated_restaurants = zomato_df.select(col('name').alias("Name of Restaurant"),\
                                      col('rating').alias("Rating"))\
                                      .orderBy(desc(col("rating")))\
                                      .limit(10)

top_rated_restaurants.show()

+--------------------+------+
|  Name of Restaurant|Rating|
+--------------------+------+
|Byg Brewski Brewi...| 4.9/5|
|Byg Brewski Brewi...| 4.9/5|
|Byg Brewski Brewi...| 4.9/5|
|Asia Kitchen By M...| 4.9/5|
|Belgian Waffle Fa...| 4.9/5|
|Belgian Waffle Fa...| 4.9/5|
|            Flechazo| 4.9/5|
|            Flechazo| 4.9/5|
|AB's - Absolute B...| 4.9/5|
|Asia Kitchen By M...| 4.9/5|
+--------------------+------+



In [17]:
top_rated_restaurants.toPandas().to_csv('zomato_output_csv/3.Top_10_rated_restaurants.csv', index=False)

## 4. Find the average, maximum, and minimum votes grouped by location

In [18]:
from pyspark.sql.functions import avg, min, max

In [19]:
votes_summary_by_location = zomato_df.groupBy("location").agg(avg("votes").alias("average_votes"),\
                                                              max("votes").alias("maximum_votes"),\
                                                              min("votes").alias("minimum_votes")
                                                              )\
                                     .orderBy(desc("average_votes"))
votes_summary_by_location.show()

+--------------------+------------------+-------------+-------------+
|            location|     average_votes|maximum_votes|minimum_votes|
+--------------------+------------------+-------------+-------------+
|Koramangala 5th B...|1321.9143135345666|        14726|            0|
|       Church Street|1092.4142259414225|         4748|           32|
|       Sarjapur Road|1043.5330578512396|        16832|            0|
|             MG Road|1015.5472972972973|         4616|            0|
|    Old Airport Road| 953.0693069306931|         9300|           26|
|       Infantry Road| 950.2666666666667|         2051|           89|
|Koramangala 3rd B...| 935.7156862745098|         2717|           75|
|        Lavelle Road| 935.5355191256831|         3730|           53|
|Koramangala 4th B...| 916.3963414634146|         7871|            0|
|         Indiranagar| 848.5818476499189|        14956|            0|
|        Brigade Road| 711.8695652173913|         4448|            0|
|Koramangala 6th B..

In [20]:
votes_summary_by_location.toPandas().to_csv('zomato_output_csv/4.Avg_max_min_Votes_by_Location.csv', index=False)

## 5. Change rate to float type by removing ‘/5’

In [21]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, FloatType

In [22]:
def convert(strr):
    if(strr == 'NEW' or strr == '-'):
        return "0"
    else:
        value = str(strr).split('/')
        value = value[0]
        return float(value)

In [23]:
convert_UDF = udf(lambda string : convert(string),StringType())

In [24]:
new_zomato_df = zomato_df.withColumn("rating", convert_UDF(col('rating')))

In [25]:
new_zomato_df = new_zomato_df.withColumn('rating', col('rating').cast(FloatType()))

In [26]:
new_zomato_df.select(col('rating')).show(10)

+------+
|rating|
+------+
|   4.1|
|   3.8|
|   3.7|
|   4.2|
|   4.2|
|   4.0|
|   3.8|
|   3.9|
|   3.7|
|   4.0|
+------+
only showing top 10 rows



In [27]:
new_zomato_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- votes: integer (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- approx_cost_two_people: integer (nullable = true)



In [28]:
new_zomato_df.toPandas().to_csv('zomato_output_csv/5.Change_rate_to_float_type_by_removing_5.csv', index=False)

## 6. Find the correlation between cost and rating of the restaurants.

In [29]:
from pyspark.sql.functions import corr

In [30]:
correlation_df = new_zomato_df.select(f.corr('approx_cost_two_people', 'rating'))

In [31]:
correlation_df.show()

+------------------------------------+
|corr(approx_cost_two_people, rating)|
+------------------------------------+
|                 0.07388867845850675|
+------------------------------------+



In [32]:
correlation_df.toPandas().to_csv('zomato_output_csv/6.Correlation_between_cost_and_rating.csv', index=False)